In [12]:
import cv2 as cv
import time
import numpy as np

confThreshold = 0.5  #Confidence threshold
inpWidth = 320       #Width of network's input image
inpHeight = 320      #Height of network's input image

Uses Opencv's deep neural networks library to run darknet with Intel GPU

In [2]:
#loads network
net = cv.dnn.readNetFromDarknet("../yolov3-tiny-obj.cfg", "../yolov3-tiny-obj_final.weights")
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_OPENCL)

Utility functions\
getOutputsNames(net): returns names of layers with unconnected outputs, i.e. output layers\
postprocess(frame, outs): applies Non maximum suppression to get results

In [3]:
def getOutputsNames(net):
    layersNames = net.getLayerNames()
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

Post process weeds out unneeded confidences. No need for NMS. Deletes all overlapping boxes. Returns position of center coordinates because that's all we need.

In [4]:
def postprocess(frame, pred):
    fHeight = frame.shape[0] #height of frame
    fWidth = frame.shape[1] #width of frame
    
    pred = np.array([x for x in pred if x[5] >= confThreshold]) #removes confidences under threshold
    
    post_pred = [] #list of all post predictions
    
    print("Frame Height: {}".format(fHeight))
    print("Frame Width: {}".format(fWidth))
    
    while pred.size > 0:
        hc_i = np.argmax(pred, axis=0)[5] #highest confidence index
        hc_row = pred[hc_i] #highest confidence row

        center_x = int(hc_row[0] * fWidth)
        center_y = int(hc_row[1] * fHeight)
        width = int(hc_row[2] * fWidth)
        post_pred.append([hc_row[5], (center_x, center_y)]) #first element to base nms off of
        
        pred = np.array([x for x in pred if checkOverlap(center_x, width, int(x[0] * fWidth))])
    
    print(post_pred)
    return post_pred

#returns boolean. True if needs to be removed
def checkOverlap(x_box, w_box, x_test):
    return not (x_box - w_box / 2) <= x_test <= (x_box + w_box / 2) #if x coordinate is overlapping deletes that stuff

Input is converted into input blob for neural network. Function for detection is written

In [5]:
def detect(net, im, confThreshold = 0.5):
    start = time.time()
    in_blob = cv.dnn.blobFromImage(im, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)

    net.setInput(in_blob)

    pred = net.forward(getOutputsNames(net))
    pred = np.array(pred) #converts into numpy array
    pred = np.concatenate([np.array(i) for i in pred]) #reshapes into 1 dimension array

    final_im = postprocess(im, pred)
    end = time.time()
    print("Total time to predict: {}".format(end-start))
    
    return final_im #returns post processed result - confidence and center_x and center_y

Row distance calculation - compares all of predicted objects and gets closest row to previous x and y

In [6]:
def closest_pred(rows, center_x, center_y):
    s_distances = [(row[1][0] - center_x)**2 + (row[1][1] - center_y)**2 for row in rows] #squared distances
    return rows[s_distances.index(min(s_distances))]

In [7]:
test_im = cv.imread("../../testImage.png") #reads image
b_info = detect(net, test_im)[0] #confidence and x,y coordinates of center

Frame Height: 562
Frame Width: 1002
[[0.9903469, (621, 236)]]
Total time to predict: 0.06746482849121094


In [8]:
#draws circle at predicted point

cv.circle(test_im, b_info[1], 10, (255, 0, 0), 4)

cv.imshow('frame',test_im)
cv.waitKey(0)
cv.destroyAllWindows()

In [21]:
cap = cv.VideoCapture("../../testVideo.mp4")
#cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()
    
all_pred = [] #plotting all predictions

#ret, frame = cap.read()
#p_x, p_y = (frame.shape[0] / 2, frame.shape[1] / 2) #sets initial position -- in middle -- works in real time
start = time.time() #for plotting purposes
while True:
    ret, frame = cap.read()

    # Capture frame-by-frame
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
        
    try:
        r = detect(net, frame, confThreshold) #returns list of balls, calculates closest distance to previous frame and uses that
        current = time.time()
        all_pred.extend([current-start] + r)
        
        p_x, p_y = r[0][1]
        print(r)
        end = time.time()
        cv.circle(frame, (p_x, p_y), 10, (255, 0, 0), 4)
        
    except:
        print("no predictions")
        
    # Display the resulting frame
    cv.imshow('frame', frame)
    if cv.waitKey(1) == ord('q'):
        break


# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

Frame Height: 720
Frame Width: 1280
[[0.9138064, (852, 459)], [0.8247548, (813, 459)]]
Total time to predict: 0.022348880767822266
[[0.9138064, (852, 459)], [0.8247548, (813, 459)]]
Frame Height: 720
Frame Width: 1280
[[0.888686, (850, 472)], [0.74486244, (809, 457)]]
Total time to predict: 0.023401975631713867
[[0.888686, (850, 472)], [0.74486244, (809, 457)]]
Frame Height: 720
Frame Width: 1280
[[0.9869121, (848, 474)], [0.9088341, (812, 457)]]
Total time to predict: 0.022602558135986328
[[0.9869121, (848, 474)], [0.9088341, (812, 457)]]
Frame Height: 720
Frame Width: 1280
[[0.9943663, (850, 477)], [0.97909534, (812, 459)]]
Total time to predict: 0.0209500789642334
[[0.9943663, (850, 477)], [0.97909534, (812, 459)]]
Frame Height: 720
Frame Width: 1280
[[0.99586624, (843, 474)], [0.959993, (799, 460)]]
Total time to predict: 0.025873184204101562
[[0.99586624, (843, 474)], [0.959993, (799, 460)]]
Frame Height: 720
Frame Width: 1280
[[0.99384594, (789, 462)], [0.80615765, (850, 475)]]
T

Frame Height: 720
Frame Width: 1280
[[0.98722535, (783, 457)], [0.5877904, (1022, 500)]]
Total time to predict: 0.027730226516723633
[[0.98722535, (783, 457)], [0.5877904, (1022, 500)]]
Frame Height: 720
Frame Width: 1280
[[0.98555017, (781, 456)], [0.94202816, (1031, 500)]]
Total time to predict: 0.020849943161010742
[[0.98555017, (781, 456)], [0.94202816, (1031, 500)]]
Frame Height: 720
Frame Width: 1280
[[0.96751773, (783, 454)]]
Total time to predict: 0.02830195426940918
[[0.96751773, (783, 454)]]
Frame Height: 720
Frame Width: 1280
[[0.94392973, (790, 454)], [0.8197208, (1050, 505)]]
Total time to predict: 0.027319908142089844
[[0.94392973, (790, 454)], [0.8197208, (1050, 505)]]
Frame Height: 720
Frame Width: 1280
[[0.9914458, (783, 455)], [0.67975396, (1062, 505)]]
Total time to predict: 0.030761003494262695
[[0.9914458, (783, 455)], [0.67975396, (1062, 505)]]
Frame Height: 720
Frame Width: 1280
[[0.99376696, (783, 457)], [0.931795, (1077, 506)]]
Total time to predict: 0.02338314

Frame Height: 720
Frame Width: 1280
[[0.9949961, (997, 484)], [0.7797645, (499, 411)]]
Total time to predict: 0.0293729305267334
[[0.9949961, (997, 484)], [0.7797645, (499, 411)]]
Frame Height: 720
Frame Width: 1280
[[0.9889703, (994, 488)], [0.8373681, (494, 413)]]
Total time to predict: 0.02225208282470703
[[0.9889703, (994, 488)], [0.8373681, (494, 413)]]
Frame Height: 720
Frame Width: 1280
[[0.9353253, (484, 416)], [0.85699475, (998, 488)]]
Total time to predict: 0.022467851638793945
[[0.9353253, (484, 416)], [0.85699475, (998, 488)]]
Frame Height: 720
Frame Width: 1280
[[0.9652141, (475, 418)], [0.95050853, (992, 492)]]
Total time to predict: 0.0267179012298584
[[0.9652141, (475, 418)], [0.95050853, (992, 492)]]
Frame Height: 720
Frame Width: 1280
[[0.9934844, (976, 487)], [0.9398158, (476, 419)]]
Total time to predict: 0.024946928024291992
[[0.9934844, (976, 487)], [0.9398158, (476, 419)]]
Frame Height: 720
Frame Width: 1280
[[0.9799158, (975, 490)], [0.97280777, (476, 421)]]
Tot

In [22]:
print(all_pred)

[0.04145407676696777, [0.9138064, (852, 459)], [0.8247548, (813, 459)], 0.10716128349304199, [0.888686, (850, 472)], [0.74486244, (809, 457)], 0.15717244148254395, [0.9869121, (848, 474)], [0.9088341, (812, 457)], 0.20376133918762207, [0.9943663, (850, 477)], [0.97909534, (812, 459)], 0.25519633293151855, [0.99586624, (843, 474)], [0.959993, (799, 460)], 0.3075542449951172, [0.99384594, (789, 462)], [0.80615765, (850, 475)], 0.3670511245727539, [0.9725738, (851, 474)], [0.96056086, (790, 458)], 0.41721606254577637, [0.95625377, (780, 462)], 0.46457529067993164, [0.68538326, (780, 461)], 0.5125181674957275, [0.9734382, (858, 478)], [0.7156771, (782, 461)], [0.506529, (352, 436)], 0.5697553157806396, [0.99086994, (855, 479)], [0.8415113, (799, 462)], 0.6252522468566895, [0.98767316, (861, 481)], [0.91181135, (786, 461)], 0.6835043430328369, [0.99597394, (868, 481)], [0.9273168, (782, 462)], 0.7394652366638184, [0.9905272, (872, 483)], [0.8756401, (777, 461)], 0.7977070808410645, [0.98068

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.plot(df.Date, df["Temperature (C)"], "g")
plt.xlabel("Date")
plt.ylabel("Temperature (C)")
plt.title("Temperature over a ten year period")
plt.show()